In [1]:
import pandas as pd

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [ ]:
APP_NAME = "Survival Analysis EDA"

try:
    sc.stop()
    spark.stop()
except:
    pass

from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

try:
    sc and spark
except (NameError, UnboundLocalError) as e:
    import findspark
    #findspark.init()
    import pyspark
    import pyspark.sql
    
    #sc = pyspark.SparkContext()
    #spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()
    
sc = SparkContext(conf = SparkConf() .set("spark.driver.maxResultSize", "60g") .set("spark.sql.execution.arrow.enabled", "true") .set('spark.sql.broadcastTimeout', 1000) .set('spark.local.dir', '/data_data/session_length/spark_tmp/') .set('spark.driver.memory', '60G') .set("spark.executor.instances", "10") .set("spark.executor.cores", 10) .set("spark.executor.memory", "20G")).getOrCreate()
spark = SparkSession(sc)
spark.sparkContext.setLogLevel("ERROR")
    

In [ ]:
sc._conf.getAll()

In [ ]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, DateType, DoubleType, IntegerType, TimestampType
from pyspark.sql.functions import count, mean, stddev_pop, min, max, lit, round, bround, pow, col, corr, lower, upper, avg, stddev, abs, log
from pyspark.sql.functions import lit, trim, rtrim, rpad, trim, coalesce
from pyspark.sql.functions import current_date, current_timestamp, date_add, date_sub, months_between, to_date
from pyspark.sql.functions import udf, col, countDistinct, lag
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, dense_rank, rank, expr

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula, Bucketizer, QuantileDiscretizer
from pyspark.ml.classification import LogisticRegression, GBTClassifier, DecisionTreeClassifier, RandomForestClassifier, MultilayerPerceptronClassifier,NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import AFTSurvivalRegression
from pyspark.ml.linalg import Vectors

from datetime import datetime
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### IMPORT MODEL DATA

In [ ]:
!head new_model_df_11_12_2018.tsv

In [ ]:
!head -n 5000 new_model_df_11_27_2018.tsv

#### Define Schema (here we assume 10 genres)

In [ ]:
session_Schema = StructType([StructField('userid', StringType(), True), 
                             StructField('session_id', IntegerType(), True), 
                             StructField('gender', StringType(), True), 
                             StructField('age', StringType(), True), 
                             StructField('country', StringType(), True), 
                             StructField('day_of_week', StringType(), True),
                             StructField('time_of_day', StringType(), True), 
                             StructField('is_holiday', IntegerType(), True), 
                             StructField('time_since_epoch', DoubleType(), True), 
                             StructField('days_since_epoch', IntegerType(), True), 
                             StructField('absence_time', IntegerType(), True),
                             StructField('genre_10_name', StringType(), True),
                             StructField('genre_9_name', StringType(), True),
                             StructField('genre_8_name', StringType(), True),
                             StructField('genre_7_name', StringType(), True),
                             StructField('genre_6_name', StringType(), True),
                             StructField('genre_5_name', StringType(), True),
                             StructField('genre_4_name', StringType(), True),
                             StructField('genre_3_name', StringType(), True),
                             StructField('genre_2_name', StringType(), True),
                             StructField('genre_1_name', StringType(), True),
                             StructField('undefined_genre_name', StringType(), True),
                             StructField('genre_10_freq', DoubleType(), True),
                             StructField('genre_9_freq', DoubleType(), True),
                             StructField('genre_8_freq', DoubleType(), True),
                             StructField('genre_7_freq', DoubleType(), True),
                             StructField('genre_6_freq', DoubleType(), True),
                             StructField('genre_5_freq', DoubleType(), True),
                             StructField('genre_4_freq', DoubleType(), True),
                             StructField('genre_3_freq', DoubleType(), True),
                             StructField('genre_2_freq', DoubleType(), True),
                             StructField('genre_1_freq', DoubleType(), True),
                             StructField('undefined_genre_freq', DoubleType(), True), 
                             StructField('session_length', StringType(), True),
                            ])


In [ ]:
#the_model_df = spark.read.format('csv').options(sep="\t").schema(session_Schema).option('header',True).load('new_model_df_11_27_2018.tsv')

#.drop('age','gender', 'genre_10','genre_9','genre_8','genre_7','genre_8','genre_7','genre_6','genre_5','genre_4','genre_3','genre_2','genre_1')

the_model_df = spark.read.format('csv').options(sep='\t').option("inferSchema", "true").option('header',True).load('new_model_df_11_27_2018.tsv')

In [ ]:
the_model_df.show(500, False)

#### Turn categorical values into integer values

In [ ]:
the_model_df.printSchema()

In [ ]:
set(the_model_df.columns)-set(['is_holiday','time_since_epoch','days_since_epoch','absence_time','genre_10_freq','genre_9_freq','genre_8_freq',
                               'genre_7_freq','genre_6_freq','genre_5_freq','genre_4_freq','genre_3_freq','genre_2_freq','genre_1_freq','undefined_genre_freq',
                               'session_length'])

In [ ]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(the_model_df) for column in list(set(the_model_df.columns)-set(['is_holiday','time_since_epoch','days_since_epoch','absence_time','genre_10_freq','genre_9_freq','genre_8_freq','country_index',
                                                                                                                                          'genre_7_freq','genre_6_freq','genre_5_freq','genre_4_freq','genre_3_freq','genre_2_freq','genre_1_freq','undefined_genre_freq','session_length',
                                                                                                                                          ])) ]

In [ ]:

pipeline = Pipeline(stages=indexers)
the_model_df_ = pipeline.fit(the_model_df).transform(the_model_df)


In [ ]:
#spark.sparkContext.setLogLevel("ERROR")
#the_model_df_.show(5)
#spark.sparkContext.setLogLevel("ERROR")

In [ ]:
the_model_DF_ = the_model_df_.select('userid','session_id','age','gender','country','country_index','day_of_week','day_of_week_index','time_of_day','time_of_day_index','is_holiday','absence_time','days_since_epoch',
                                    'genre_10_name','genre_9_name','genre_8_name', 'genre_7_name','genre_6_name','genre_5_name','genre_4_name','genre_3_name','genre_2_name','genre_1_name','undefined_genre_name',
                                    'genre_10_freq','genre_9_freq','genre_8_freq', 'genre_7_freq','genre_6_freq','genre_5_freq','genre_4_freq','genre_3_freq','genre_2_freq','genre_1_freq','undefined_genre_freq','session_length')

#the_model_DF_ = the_model_df_

#the_model_DF_ = the_model_df_.select('userid','session_id','country','country_index','day_of_week','day_of_week_index','time_of_day','time_of_day_index','is_holiday','absence_time','days_since_epoch','session_length')

In [ ]:
the_model_DF_.printSchema()

In [ ]:
false_cols = the_model_DF_.columns

In [ ]:
#false_cols.append('age')

In [ ]:
false_cols

In [ ]:
def set_df_columns_nullable(spark, df, column_list, nullable=True):
    for struct_field in df.schema:
        if struct_field.name in column_list:
            struct_field.nullable = nullable
    df_mod = spark.createDataFrame(df.rdd, df.schema)
    return df_mod

In [ ]:
_the_model_DF = set_df_columns_nullable(spark,the_model_DF_,false_cols)

In [ ]:
the_model_DF = _the_model_DF
#.withColumn("age_", _the_model_DF["age"].cast("double"))
#.drop('age').withColumnRenamed('age_','age')

In [ ]:
the_model_DF.printSchema()

In [ ]:
the_model_DF.orderBy('userid','session_id').show(5000, False)

In [ ]:
the_model_DF.select('userid').distinct().count()

In [ ]:
userSession_df = the_model_DF

In [ ]:
userSession_df.show()

In [ ]:
userSession_df.select('userid').distinct().count()

### EDA

#### Explore Distribution of Session Lengths

In [ ]:
userSession_df.columns

In [ ]:
plot_df = userSession_df.toPandas().dropna()

In [ ]:
plot_df2 = plot_df[['day_of_week','time_of_day','session_length']].dropna()

In [ ]:
g = sns.PairGrid(plot_df2, vars=[ 'time_of_day', 'session_length'],hue='time_of_day', palette='RdBu_r')
g.map(plt.scatter, alpha=0.8)
g.add_legend();

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(36, 15)
sns.countplot(plot_df['session_length'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
plt.show()

In [ ]:
userSession_df = userSession_df.withColumn('log_sess', round(log("session_length"),2)).drop('session_length').withColumnRenamed('log_sess','session_length')


In [ ]:
userSession_df.show(10, False)

In [ ]:
#w = Window.partitionBy("userid").orderBy("window").rowsBetween(Window.unboundedPreceding,Window.currentRow)
#(user_session_with_window_df.withColumn("cumSum", sum("session_length").over(w)).show())

In [ ]:
plot_log_df = userSession_df.toPandas().dropna()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 15)
sns.countplot(plot_log_df['session_length'])

In [ ]:
with sns.axes_style(style='ticks'):
    g = sns.factorplot("day_of_week", "session_length", "time_of_day", data=plot_log_df, kind="box", size=4, aspect=4)
    g.set_axis_labels("Day of Week", "Session Length")

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 12)
sns.countplot(plot_df['day_of_week'].dropna())

In [ ]:
sess_tod_df = userSession_df.groupBy('time_of_day').agg(count("time_of_day").alias("num_sessions"))
sess_tod_df.show()

In [ ]:
timeofday    = list([x[0] for x in pd.Series(sess_tod_df.select('time_of_day').collect())])
num_sessions = list([y[0] for y in pd.Series(sess_tod_df.select('num_sessions').collect())])

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15,12)
sns.countplot(plot_df['time_of_day'])

In [ ]:
sess_tod_sl_df = userSession_df.groupBy('time_of_day').agg(round(avg("session_length"),2).alias("avg_sess_length"))
sess_tod_sl_df.show()

In [ ]:
avg_sess_len = list([z[0] for z in pd.Series(sess_tod_sl_df.select('avg_sess_length').collect())])

In [ ]:
avg_sess_len

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15,12)


In [ ]:
#plt.bar()

In [ ]:

sns.pairplot(plot_df2, hue='time_of_day', size=4.0)
plt.show()

In [ ]:
sns.lmplot("age", "session_length", data=plot_log_df, hue='time_of_day', fit_reg=False, size=4.0)

In [ ]:
#sns.jointplot("session_length", "age", data=plot_log_df.dropna(), kind='reg');

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(36, 24)
sns.countplot(plot_log_df['age'])

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(36, 24)
sns.barplot(x="age", y="session_length", hue="gender", data=plot_log_df)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
sns.barplot(x="day_of_week", y="session_length", hue="gender", data=plot_log_df)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
sns.barplot(x="time_of_day", y="session_length", hue="gender", data=plot_log_df)

In [ ]:
cols = ['age']
fig, ax = plt.subplots()
fig.set_size_inches(24, 12)
#for col in cols:
#    sns.kdeplot(plot_log_df[col], shade=True)

In [ ]:
#sns.kdeplot(plot_log_df.session_length[plot_log_df.gender=='M'], label='men', shade=True)
#sns.kdeplot(plot_log_df.session_length[plot_log_df.gender=='W'], label='women', shade=True)
#plt.xlabel('split_frac');

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(36, 15)
sns.countplot(plot_log_df['country'], order = plot_log_df['country'].value_counts().index)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.show()

In [ ]:
with sns.axes_style(style='ticks'):
    g = sns.factorplot("day_of_week", "session_length", "time_of_day", data=plot_df, kind="box", size=6, aspect=12)
    g.set_axis_labels("Day of Week", "Session Length")

In [ ]:
with sns.axes_style('white'):
    g = sns.factorplot("day_of_week", data=plot_log_df, aspect=8.0, kind='count',
                       hue='time_of_day', size=6.0)
    g.set_ylabels('Session Length')

## CLASSIFICATION

In [ ]:
data = userSession_df.select('session_length').rdd.map(lambda row: row.session_length).collect()
data[:10]

In [ ]:
data = pd.DataFrame(data,columns=['session_length']).iloc[:,0]

In [ ]:
data.head(10)

In [ ]:
data.describe()

In [ ]:
deciles = list(np.percentile(data, np.arange(0, 100, 10)))
deciles

#### Add Quantile Bucketized Column

In [ ]:
userSession_df.show()

In [ ]:
userSession_df.select('userid').distinct().count()

In [ ]:
num_windows = int(userSession_df.select('session_length').distinct().count())
num_windows

In [ ]:
num_buckets = 2.5*len(deciles)
num_buckets

In [ ]:
session_length_bucketizer = QuantileDiscretizer(numBuckets=num_buckets, inputCol="session_length", outputCol="decile", relativeError=0.01, handleInvalid="error")
bucketizer = session_length_bucketizer.fit(userSession_df)
user_session_with_window_bucketized_df = bucketizer.setHandleInvalid("skip").transform(userSession_df)
user_session_with_window_bucketized_df.show()


In [ ]:
user_session_with_window_bucketized_df.select('userid').distinct().count()

In [ ]:
user_session_with_window_bucketized_df.printSchema()

In [ ]:
decile_dict = {1.0:"<5_mins",2.0:"<5_mins",3.0:"<10_mins",4.0:"<20_mins",5.0:"<30_mins",6.0:"<40_mins",7.0:"<1_hour",8.0:"<1.5_hours",9.0:"<2_hours",10.0:">3_hours"}

In [ ]:
decile_dict

In [ ]:
def get_description(decile):
    try:
        return decile_dict[decile]
    except:
        return None

In [ ]:
get_description(1.0)

In [ ]:
get_desc = udf(lambda bucket: get_description(bucket), returnType=StringType())

In [ ]:
user_session_with_window_bucketized_descr_df = user_session_with_window_bucketized_df.withColumn('description', get_desc('decile'))

In [ ]:
user_session_with_window_bucketized_descr_df.show(20)

In [ ]:
user_session_with_window_bucketized_descr_df.select('userid').distinct().count()

In [ ]:
int("user_000002".split("_")[1].lstrip('0'))

In [ ]:
def get_user_num(userid):
    return str(userid.split("_")[1].lstrip("0"))
get_user_num_udf = udf(lambda userid: get_user_num(userid), IntegerType())

In [ ]:
user_session_with_window_bucketized_descr_df = user_session_with_window_bucketized_descr_df.withColumn('user_num', get_user_num_udf('userid'))

In [ ]:
user_session_with_window_bucketized_descr_df.show(10, False)

In [ ]:
train_data = None
dev_data = None
test_data = None

In [ ]:
df = user_session_with_window_bucketized_descr_df


#df.select($"userid",concat_ws(":",$"genre_10",$"genre_9",$"genre_9",$"genre_8",$"genre_7",$"genre_6",$"genre_5",$"genre_4",$"genre_3",$"genre_2",$"genre_1").as("new_col")).groupBy($"userid").agg(collect_set($"new_col")).show()

### ASSIGN USERS TO CLUSTERS BASED ON TIME_OF_DAY VARIANCES IN SESSION_LENGTH

In [ ]:
user_df = user_session_with_window_bucketized_df.select('userid','session_id','country','time_of_day','day_of_week','is_holiday','session_length' )

In [ ]:
user_df.show()

In [ ]:
user_var_df = user_df.select("userid","time_of_day","session_length").groupBy('userid','time_of_day').agg(round(avg("session_length"),1).alias('avg_time_of_day'),round(stddev("session_length"),1).alias('std_time_of_day'))

In [ ]:
user_var_df.orderBy('userid').show(10, False)

In [ ]:
#user_var_df.pivot('time_of_day')


#user_var_df.withColumn("fg", F.row_number().over(Window().partitionBy('userid').groupBy('userid').pivot('fg').sum('avg_time_of_day', 'std_time_of_day').show()
                                                 
_user_var_df = user_var_df.select('userid','time_of_day','avg_time_of_day').na.fill(0.0).groupBy("userid").pivot("time_of_day").agg(max("avg_time_of_day")).drop('Null').dropna()

_user_var_df.show()


In [ ]:
df = _user_var_df

In [ ]:

new_df = df.withColumn('total', round(sum(df[col] for col in df.columns[1:])))
#my_schema = StructType([StructField(df.columns[0], StringType(), True)] + [StructField(col, DoubleType(), True) for col in df.columns[1:]])

In [ ]:
new_df.show()

In [ ]:
new_df_cols = new_df.columns[1:-1]

In [ ]:
new_df_cols

In [ ]:
num_buckets

In [ ]:
#new_bucketized_df.show(500)

In [ ]:
#num_buckets = 10

In [ ]:
new_df.select('Afternoon').describe()

In [ ]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

stages = []

for col in new_df_cols:
    col_assembler = VectorAssembler(inputCols= [col], outputCol = col+"_feature")
    col_scaler = StandardScaler(inputCol=col+"_feature", outputCol=col+"_scaled",withStd=True, withMean=True)
    stages.append(col_assembler)
    stages.append(col_scaler)
    
print(stages)
col_pipeline = Pipeline(stages=stages)
scaled = col_pipeline.fit(new_df)
new_df_scaled = scaled.transform(new_df)
type(new_df_scaled)

In [ ]:
new_df_scaled.show(20, False)

In [ ]:
new_df_scaled.columns

In [ ]:
scaled_cols = ['userid',
 'Afternoon_scaled',
 'Dawn_scaled',
 'Evening_scaled',
 'LateAfternoon_scaled',
 'LateEvening_scaled',
 'LateMorning_scaled',
 'Lunch_scaled',
 'Morning_scaled',
 'Night_scaled']

In [ ]:
new_df_scaled_ = new_df_scaled.select(scaled_cols)
new_df_scaled_.show(20, False)

In [ ]:
[ 'round(df[\"'+col +"\"],2).alias(\"new_"+col+"\")" for col in scaled_cols]

In [ ]:
new_df_scaled_.columns

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors



def get_tod_cluster():
    global user_tod_cluster_df
    global ug_df
    global stringIndexer
    global ug_transformed
    
    ug_df = new_df_scaled_
    
    categoricalColumns = ["userid"]
    cols = new_df_scaled_.columns
    stages =[]
    
    for categoricalCol in categoricalColumns:
        stringIndexer  = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol+"classVec"])
        stages += [stringIndexer,encoder]

    numericCols = ['Afternoon_scaled','Dawn_scaled', 'Evening_scaled', 'LateAfternoon_scaled','LateEvening_scaled','LateMorning_scaled', 'Lunch_scaled', 'Morning_scaled','Night_scaled']
    assemblerInputs = [c + "classVec" for c in categoricalColumns]

    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
    
    stages += [assembler]
    pipeline = Pipeline(stages=stages)
    pipelineModel = pipeline.fit(ug_df)
    ug_df = pipelineModel.transform(ug_df)
    selectedcols = cols + ["features"]
    print(selectedcols)
    ug_df = ug_df.select(selectedcols)


    ug_df.show(20, False)

In [ ]:
get_tod_cluster()

In [ ]:
kmeans = KMeans(k=10, seed=1)
model = kmeans.fit(ug_df)

ug_tr_df = model.transform(ug_df)
    
ug_tr_df = ug_tr_df.withColumnRenamed('userid','userid_tod')

ug_tr_df.show(1000, False)


In [ ]:
_user_var_df.columns

In [ ]:
user_tod_cluster_df = None

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors



def get_tod_cluster():
    global user_tod_cluster_df
    global ug_df
    global stringIndexer
    global ug_transformed
    
    ug_df = _user_var_df
    
    categoricalColumns = ["userid"]
    cols = _user_var_df.columns
    stages =[]
    
    for categoricalCol in categoricalColumns:
        stringIndexer  = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol+"classVec"])
        stages += [stringIndexer,encoder]

    numericCols = ["Afternoon","Dawn","Evening","LateAfternoon","LateEvening","LateMorning","Lunch","Morning","Night"]
    assemblerInputs = [c + "classVec" for c in categoricalColumns]

    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
    
    stages += [assembler]
    pipeline = Pipeline(stages=stages)
    pipelineModel = pipeline.fit(ug_df)
    ug_df = pipelineModel.transform(ug_df)
    selectedcols = ["features"] + cols
    ug_df = ug_df.select(selectedcols)


    ug_df.select('userid','features').show(20, False)
    

In [ ]:
get_tod_cluster()

In [ ]:
    
kmeans = KMeans(k=10, seed=1)
model = kmeans.fit(ug_df.select('features'))

In [ ]:
ug_tr_df = model.transform(ug_df)
    
ug_tr_df = ug_tr_df.select('userid','prediction').withColumnRenamed('userid','userid_tod')

ug_tr_df.show(10, False)

In [ ]:
ugTR_df = ug_tr_df.withColumnRenamed('prediction','tod_cluster') 
ugTR_df.show()

## ASSIGN USERS TO CLUSTERS BASED ON MUSIC TASTES (GENRE)

In [ ]:
user_genres_df = user_session_with_window_bucketized_df.select("userid","genre_10_name","genre_9_name","genre_8_name","genre_7_name","genre_6_name","genre_5_name","genre_4_name","genre_3_name","genre_2_name","genre_1_name")
user_genres_df.show(10)

In [ ]:
usGN_df = user_genres_df.select('userid','genre_10_name').groupBy('userid','genre_10_name').count().orderBy(['userid','count'], ascending=[1, 0])
usGN_df.show(40)

In [ ]:
window = Window.partitionBy(df['userid']).orderBy(usGN_df['count'].desc())

#us_GN_df = usGN_df.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 10).orderBy(['userid','rank'],ascending=[1,1]).select('userid','genre_10_name')

#us_GN_df.show(30)

In [ ]:
user_cluster_df = None
assembler = None
ug_df = None
ug_transformed = None

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors



def get_top_genre_cluster():
    global user_cluster_df
    global ug_df
    global assembler
    global stringIndexer
    global ug_transformed
    global ug_gn_df
    
    ug_df = usGN_df.select('userid','genre_10_name')
    
    categoricalColumns = ["userid","genre_10_name"]
    cols = usGN_df.select('userid','genre_10_name').columns
    stages =[]
    
    for categoricalCol in categoricalColumns:
        stringIndexer  = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol+"classVec"])
        stages += [stringIndexer,encoder]

    numericCols = []
    assemblerInputs = [c + "classVec" for c in categoricalColumns]
    #assemblerInputs = numericCols
    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
    
    stages += [assembler]
    pipeline = Pipeline(stages=stages)
    pipelineModel = pipeline.fit(ug_df)
    ug_df = pipelineModel.transform(ug_df)
    selectedcols = ["features"] + cols
    ug_gn_df = ug_df.select(selectedcols)


    ug_gn_df.select('userid','features').show(20, False)
    

    

In [ ]:
 get_top_genre_cluster()

In [ ]:
kmeans = KMeans(k=30, seed=1)
model = kmeans.fit(ug_gn_df.select('features'))

ug_gn_df = model.transform(ug_gn_df)
    
#ug_df = ug_gn_df.select('userid','prediction')

ug_df = ug_gn_df

ug_df.show(10, False)


In [ ]:
ugGN_df = ug_df.withColumnRenamed('prediction','genre_cluster').withColumnRenamed('userid','userid_gn')
ugGN_df.show()

In [ ]:
user_session_with_window_bucketized_descr_df.show()

In [ ]:
ugGN_df.show(1000)

In [ ]:
ugTR_df.orderBy('userid_tod').show(1000)

In [ ]:
_the_df = user_session_with_window_bucketized_descr_df.select("userid", "country","day_of_week", "time_of_day","is_holiday", "absence_time","days_since_epoch","session_length","decile")

In [ ]:
#getTOD_cluster_id = udf(lamb)

In [ ]:

#t = ugTR_df.alias('t')
g = ugGN_df.alias('g')
d = _the_df.alias('d')
#join_condition = [ (d.userid == t.userid_tod) ]
join_condition = [ (d.userid == g.userid_gn) ]
#bucketized_DF = d.join(t, join_condition, 'inner').drop('userid_tod')
bucketized_DF = d.join(g, join_condition, 'inner').drop('userid_gn')

bucketized_DF.show(2000,False)

In [ ]:
#user_session_with_window_bucketized_descr_df.show()

In [ ]:
from pyspark.sql.functions import percent_rank

def get_train_dev_test_data():
    print("==================================================================================================================")
    print("============================================     CREATE TRAIN/DEV/TEST SETS      =================================")
    print("==================================================================================================================\n\n")
    global train_data, dev_data, test_data
    global user_session_with_window_df
    
    print("SESSION_DATA_WITH_WINDOW_BUCKETIZED_DESCR_DF.USERID.COUNT():\t{}".format(user_session_with_window_bucketized_descr_df.select("userid").distinct().count()))
    #session_data = user_session_with_window_bucketized_descr_df.select("userid","gender","age","country", "day_of_week", "time_of_day","is_holiday","genre_10_name","genre_9_name","genre_8_name","genre_7_name","genre_6_name","genre_5_name","genre_4_name","genre_3_name","genre_2_name","genre_1_name","genre_10_freq","genre_9_freq","genre_8_freq","genre_7_freq","genre_6_freq","genre_5_freq","genre_4_freq","genre_3_freq","genre_2_freq","genre_1_freq","absence_time","days_since_epoch","session_length","decile").na.drop()
    session_data = bucketized_DF
    
    print("SESSION_DATA.USERID.COUNT():\t{}".format(session_data.select("userid").distinct().count()))
    categoricalColumns = ["userid","country","day_of_week", "time_of_day"]
    cols = session_data.columns
    stages =[]
    for categoricalCol in categoricalColumns:
        stringIndexer  = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol+"classVec"])
        stages += [stringIndexer,encoder]
    label_stringIdx = StringIndexer(inputCol="decile", outputCol="label")
    stages += [label_stringIdx]
    numericCols = ["is_holiday","absence_time","days_since_epoch","session_length","decile",'genre_cluster']
    assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
    stages += [assembler]
    pipeline = Pipeline(stages=stages)
    pipelineModel = pipeline.fit(session_data)
    session_data = pipelineModel.transform(session_data)
    selectedcols = cols + ["features","label"] 
    session_data = session_data.select(selectedcols)
    print("************************************       DATA BEFORE TRAIN/DEV/TEST SPLIT     *************************************")
    #session_data.show(5,False)
    print("************************************           Splitting Train/Test data        *************************************")
    #train_data, dev_data, test_data = session_data.randomSplit([0.65, 0.2, 0.15])
    
    print("SESSION_DATA.COUNT():\t{}".format(session_data.count()))
    print("SESSION_DATA.USERID.COUNT():\t{}".format(session_data.select("userid").distinct().count()))

    session_data_df = session_data.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("userid"))).drop('decile')
    train_data = session_data_df.where("rank <= .65").drop("rank")
    #train_data.show()
    dev_data = session_data_df.where(("rank > .65") and ("rank < 0.799")).drop("rank")
    #dev_data.show()
    test_data = session_data_df.where("rank > .80").drop("rank")
    #test_data.show()

In [ ]:
get_train_dev_test_data()

In [ ]:
train_data.printSchema()

In [ ]:
train_data.show(5,False)

In [ ]:
dev_data.show(5,False)

In [ ]:
test_data.show(5,False)

### BASELINE DL model with Multinomial Naive Bayes

In [ ]:
smoothings  = [0.2,0.4,0.6,0.8,1.0]

In [ ]:
mnbModel = None

In [ ]:
results_mnb = []

def mnb_classifier(smoothing):
    global results_mnb
    global mnbModel
    global lr_results
    global train_data, dev_data, test_data
    mnb = NaiveBayes(smoothing=smoothing, modelType="multinomial")
    #print("---------------------------------  Fitting Model to TRAIN  ---------------------------")
    mnbModel = mnb.fit(train_data)
    #print("---------------------------------  Generating Predictions  ---------------------------")
    predictions = mnbModel.transform(dev_data)
    selected = predictions.select("label", "prediction", "userid","country","is_holiday","country", "day_of_week", "time_of_day","absence_time","days_since_epoch",'genre_cluster')
    evaluator = MulticlassClassificationEvaluator().setMetricName('accuracy').setLabelCol('label').setPredictionCol("prediction")
    #print("---------------------------------  Evaluating Model     ------------------------------")
    dev_result = evaluator.evaluate(mnbModel.transform(dev_data))
    test_result = evaluator.evaluate(mnbModel.transform(test_data))
    #result.select("features","label","prediction").show(5,False)
    mnb_results.update({smoothing:[dev_result,test_result]})


In [ ]:


def eval_mnb_classifier():
    print("###########################################################################################")
    print("############################     MULTINOMIAL NAIVE BAYES CLASSIFIER      ##################")
    print("###########################################################################################")
    print("\tSMOOTHING\tACCURACY\n\t\t\tDEV\t\tTEST")
    for smoothing in smoothings:
        mnb_classifier(smoothing)
        print("\t{}\t\t{}\t{}".format(smoothing, mnb_results[smoothing][0],mnb_results[smoothing][1]))
        

In [ ]:
mnb_results = dict()

#eval_mnb_classifier()

### BASELINE DL model with Multinomial Logistic Regression

In [ ]:
lrModel = None

In [ ]:
def logistic_classifier(regparam, elasticnetparam):
    global lrModel
    global lr_results
    global train_data, dev_data, test_data
    lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10, regParam=regparam, elasticNetParam=elasticnetparam)
    #print("---------------------------------  Fitting Model to TRAIN  ---------------------------")
    lrModel = lr.fit(train_data)
    #print("---------------------------------  Generating Predictions  ---------------------------")
    predictions = lrModel.transform(dev_data)
    selected = predictions.select("label", "prediction", "userid", "country","is_holiday", "day_of_week", "time_of_day","absence_time","days_since_epoch","genre_cluster")
    
    evaluator = MulticlassClassificationEvaluator().setMetricName('accuracy').setLabelCol('label').setPredictionCol("prediction")
    #print("---------------------------------  Evaluating Model     ------------------------------")
    dev_result  = evaluator.evaluate(lrModel.transform(dev_data))
    test_result = evaluator.evaluate(lrModel.transform(test_data))
    #result.select("features","label","prediction").show(5,False)
    lr_results.update({elasticnetparam:{regparam:[dev_result,test_result]}})
    #print("MULTINOMIAL LOGISTIC REGRESSION RESULTS:\t{}".format(lr_results))

In [ ]:
regparams        = [ 0.0001,  0.001, 0.1, 0.3]
elasticnetparams = [0.0001,  0.001, 0.1, 0.3]

In [ ]:
def eval_mlr_classifier():
    print("###########################################################################################")
    print("###############################         LOGISTIC REGRESSION         #######################")
    print("###########################################################################################")
    print("\tREGPARAM\tELASTICNETPARAM\t\tACCURACY\n\t\t\t\t\t\tDEV\t\t\tTEST")
    for elasticnetparam in elasticnetparams:
        for regparam in regparams:
            logistic_classifier(regparam, elasticnetparam)
            #print(lr_results)
            #print(lr_results[elasticnetparam][regparam])
            dev,test  = lr_results[elasticnetparam][regparam]
            print("\t{}\t\t{}\t\t\t{}\t\t{}".format(regparam,elasticnetparam,dev,test))
        

In [ ]:
lr_results = {}
eval_mlr_classifier()

In [ ]:
results = "	0.0001		0.0001			0.886543634124		0.862036525917	0.001		0.0001			0.886580966992		0.862090949378	0.1		0.0001			0.883013011394		0.873092263316	0.3		0.0001			0.868542080365		0.865504077872	0.0001		0.001			0.886547189635		0.862036525917	0.001		0.001			0.893503547511		0.827158862083	0.1		0.001			0.882447685095		0.870378865038	0.3		0.001			0.868991852546		0.87603113022	0.0001		0.1			0.854085371382		0.809867750989	0.001		0.1			0.85304716208		0.847800903429	0.1		0.1			0.873478907818		0.874802714953	0.3		0.1			0.834254507055		0.879164366628	0.0001		0.3			0.854261369192		0.809735579726	0.001		0.3			0.883389895592		0.849550228967	0.1		0.3			0.830195890896		0.872579127825	0.3		0.3			0.65494296071		0.72479610639"

In [ ]:
results.replace("\t",",").replace(",,,",",").replace(",,",",")

In [ ]:
results = [
    [0.0001,0.0001,0.886543634124,0.862036525917],
    [0.001,0.0001,0.886580966992,0.862090949378],
    [0.1,0.0001,0.883013011394,0.873092263316],
    [0.3,0.0001,0.868542080365,0.865504077872],
    [0.0001,0.001,0.886547189635,0.862036525917],
    [0.001,0.001,0.893503547511,0.827158862083],
    [0.1,0.001,0.882447685095,0.870378865038],
    [0.3,0.001,0.868991852546,0.87603113022],
    [0.0001,0.1,0.854085371382,0.809867750989],
    [0.001,0.1,0.85304716208,0.847800903429],
    [0.1,0.1,0.873478907818,0.874802714953],
    [0.3,0.1,0.834254507055,0.879164366628],
    [0.0001,0.3,0.854261369192,0.809735579726],
    [0.001,0.3,0.883389895592,0.849550228967],
    [0.1,0.3,0.830195890896,0.872579127825],
    [0.3,0.3,0.65494296071,0.72479610639]]

results_df = pd.DataFrame(results, columns=['regparam','elasticparam','dev_data','test_data'])
results_df

In [ ]:
dev_error  = [1-x for x in results_df['dev_data'].tolist()][::-1]
test_error = [1-x for x in results_df['test_data'].tolist()][::-1]

In [ ]:
fig, ax = plt.subplots()
x = [x for x in range(0,16)]
ax.plot(x, dev_error, lw=3, label='Dev')
ax.plot(x, test_error, lw=3, label='Test')


ax.grid(True)
ax.legend(frameon=False)
plt.xlabel('Iteration')
plt.xlabel('Error')


In [ ]:
trainingSummary = lrModel.summary
""
objectiveHistory = trainingSummary.objectiveHistory
plt.plot(objectiveHistory)
plt.ylabel('Objective Function')
plt.xlabel('Iteration')
plt.show()

In [ ]:
predictions = lrModel.transform(dev_data)
predictions.select("features","label","prediction").show(10, False)

In [ ]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
predictions.select('userid','country','time_of_day','absence_time','is_holiday','genre_cluster','prediction').sample(0.001,100).show(30, False)

In [ ]:
class_temp  = predictions.select("label").groupBy("label").count().sort('count', ascending=False).toPandas()
class_temp

In [ ]:
class_names = class_temp['label']

In [ ]:
class_names

### Manually Create Confusion Matrix

In [ ]:
df = predictions.select('label','prediction')
df.show()

In [ ]:
print("\tLABEL\tLabels\tPredictions")
for label in class_names:
    num_labels = df[df['label'] == label].count()
    num_correct_preds = df[(df['label'] == label) & (df['prediction'] == label)].count()
    print("\t{}\t{}\t{}".format(label,num_labels,num_correct_preds))

In [ ]:
predictions_rdd = predictions.select('prediction','label').rdd.map(tuple)

In [ ]:
predictions_rdd.take(5)

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics



In [ ]:
metrics = MulticlassMetrics(predictions_rdd.map(tuple))

In [ ]:


# Overall statistics
precision = metrics.precision()
recall = metrics.recall()
f1Score = metrics.fMeasure()
print("Summary Stats")
print("Precision = \t\t%s" % precision)
print("Recall = \t\t%s" % recall)
print("F1 Score = \t\t%s" % f1Score)

# Statistics by class
#labels = data.map(lambda lp: lp.label).distinct().collect()
labels = class_names
for label in sorted(labels):
    print("Class %s precision = \t%s" % (label, metrics.precision(label)))
    print("Class %s recall = \t%s" % (label, metrics.recall(label)))
    print("Class %s F1 Measure = \t%s" % (label, metrics.fMeasure(label, beta=1.0)))

# Weighted stats
print("Weighted recall = \t%s" % metrics.weightedRecall)
print("Weighted precision = \t%s" % metrics.weightedPrecision)
print("Weighted F(1) Score = \t%s" % metrics.weightedFMeasure())
print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)

In [ ]:
m = metrics.confusionMatrix()

In [ ]:
from pyspark.mllib.linalg import Vectors, DenseMatrix 

import seaborn as sns
sns.set() 


def display_cm(m):
  a = m.toArray().astype(np.float)
  row_sums = a.sum(axis=1)
  percentage_matrix = a.astype(np.float) / row_sums[:, np.newaxis]
  print(percentage_matrix)
  plt.figure(figsize=(3, 3))
  sns.heatmap(percentage_matrix, annot=True,  fmt='.2f', xticklabels=['0' ,'1','2'], yticklabels=['0' ,'1','2']);
  plt.title('Confusion Matrix');

In [ ]:
display_cm(m)

### BASELINE DL model with an generic MLP Classifier

In [ ]:
train_data.printSchema()

In [ ]:
def mlp_classifier():
    print("###########################################################################################")
    print("##################################       MLP CLASSIFIER      ##############################")
    print("###########################################################################################")
    global mlp_results
    global train_data, dev_data, test_data
    layer_1 = train_data.schema["features"].metadata["ml_attr"]["num_attrs"]
    layer_2 = layer_1*4
    layer_3 = int(layer_2/2)
    layer_4 = num_buckets
    layers = [layer_1, layer_2, layer_3, layer_4]
    print("LAYERS: \t{}".format(layers))
    print("---------------------------------  Training MLP Model        ------------------------------")
    trainer = MultilayerPerceptronClassifier(maxIter=5, layers=layers, blockSize=128)
    print("---------------------------------  Fitting Model to TRAIN    ------------------------------")
    mlp_model = trainer.fit (train_data)        
    print("---------------------------------  Testing Model against DEV ------------------------------")
    predictions = mlp_model.transform(dev_data)
    selected = predictions.select("label", "prediction", "userid","country", "day_of_week", "time_of_day", "is_holiday",
                                  "previous_duration","absence_time","is_holiday","genre_cluster")
    print("evaluator = MulticlassClassificationEvaluator().....")
    evaluator = MulticlassClassificationEvaluator().setMetricName('accuracy').setLabelCol('label').setPredictionCol("prediction")
    print("result = evaluator.evaluate(mlp_model.transform(dev_data))")
    result = evaluator.evaluate(mlp_model.transform(dev_data))
    mlp_results.append(result)
    print("MLP CLASSIFIER RESULTS:\t{}".format(mlp_results))

In [ ]:
#mlp_results = []
#mlp_classifier()

### BASELINE Model with Decision Tree Classifier

In [ ]:
dt_results = dict()
maxBins = 100


In [ ]:
def decisionTree_classifier(max_depth):
    global dt_results
    global train_data, dev_data, test_data
    dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=max_depth, maxBins=100)
    dtModel = dt.fit(train_data)
    #print("numNodes = ", dtModel.numNodes)
    #print("depth = ", dtModel.depth)
    evaluator = MulticlassClassificationEvaluator().setMetricName ('accuracy').setPredictionCol ('prediction').setLabelCol ('label')
    #print("result = evaluator.evaluate(dfModel.transform(dev_data))")
    result = evaluator.evaluate(dtModel.transform(dev_data))
    #print("dt_results.append(result)")
    dt_results.update({max_depth:result})
    #print("DECISION TREE CLASSIFIER RESULTS:\t{}".format(dt_results))

In [ ]:
def eval_dt_classifier():
    print("###########################################################################################")
    print("###############################          DECISION TREE              #######################")
    print("###########################################################################################")
    print("\tNUM_TREES\tMAX_TREE_DEPTH\t\tACCURACY")
    global dt_results
    for max_tree_depth in max_tree_depths:
        decisionTree_classifier(max_tree_depth)
        print("\t\t\t\t{}\t\t{}".format(max_tree_depth,dt_results[max_tree_depth]))
        

In [ ]:
max_tree_depths = [1,2,3]

In [ ]:
dt_results = dict()
#eval_dt_classifier()

### BASELINE Model with RandomForest Classifier

In [ ]:
rfModel = None
def randomForest_classifier(num_tree, max_tree_depth):
    global rf_results
    global rfModel
    global train_data, dev_data, test_data
    #rf = RandomForestClassifier(labelCol="label", featuresCol="features", maxDepth=20, maxBins=100)
    rf = RandomForestClassifier(numTrees=num_tree, featureSubsetStrategy="auto", impurity="gini", labelCol="label", featuresCol="features", maxDepth=max_tree_depth, maxBins=100)
    
    #print("---------------------------------  Fitting Model to Train     ------------------------------")
    rfModel = rf.fit(train_data)
    #print("---------------------------------  Transforming Dev           ------------------------------")
    predictions = rfModel.transform(dev_data)
    selected = predictions.select("label", "prediction", "userid", "is_holiday","country", "day_of_week", "time_of_day",
                                  "absence_time","days_since_epoch","genre_cluster")
    #print("---------------------------------     Evaluating Model        ------------------------------")
    evaluator   = MulticlassClassificationEvaluator().setMetricName('accuracy').setPredictionCol ('prediction').setLabelCol ('label')
    dev_result  = evaluator.evaluate(rfModel.transform(dev_data))
    test_result = evaluator.evaluate(rfModel.transform(test_data))
    #print("---------------------------------  Summarizing Results        ------------------------------")
    rf_results.update({num_tree:{max_tree_depth:[dev_result, test_result]}})
    #print("RANDOM FOREST CLASSIFIER RESULTS:\t{}".format(rf_results))

In [ ]:
def eval_rf_classifier():
    print("###########################################################################################")
    print("###############################          RANDOM FOREST              #######################")
    print("###########################################################################################")
    print("\tNUM_TREES\tMAX_TREE_DEPTH\t\tACCURACY\n\t\t\t\t\t\tDEV\t\t\tTEST")
    for num_tree in num_trees:
        for max_tree_depth in max_tree_depths:
            randomForest_classifier(num_tree, max_tree_depth)
            dev,test = rf_results[num_tree][max_tree_depth]
            print("\t{}\t\t{}\t\t\t{}\t\t{}".format(num_tree, max_tree_depth,dev,test))
        

In [ ]:
!whoami

In [ ]:
max_tree_depths = [ 5, 10, 15, 20, 30]
num_trees       = [ 5, 10, 15, 20, 30]

In [ ]:
rf_results = dict()

eval_rf_classifier()

In [ ]:
deciles = 25



In [ ]:

deciles = 25

###########################################################################################
###############################          RANDOM FOREST              #######################
###########################################################################################
	NUM_TREES	MAX_TREE_DEPTH		ACCURACY
						DEV			TEST
	5		5			0.674507161689		0.688425684764
	5		10			0.800491371663		0.74468399406
	5		15			0.851774289031		0.831116225189
	5		20			0.880554375323		0.860660389827
	5		30			0.934310150807		0.950482425109
	10		5			0.708462294691		0.796448480419
	10		10			0.891016467351		0.846829055908
	10		15			0.929309324151		0.899736434952
	10		20			0.960526713445		0.947543558206
	10		30			0.991724547428		0.997356574743
	15		5			0.859086198038		0.797062688052
	15		10			0.909874899335		0.869982351249
	15		15			0.933277274772		0.904121410967
	15		20			0.971532798703		0.962813226456
	15		30			0.993955630774		0.998997053358
	20		5			0.833157631816		0.78075897404
	20		10			0.909764678484		0.865675123036
	20		15			0.944830908771		0.921956756673
	20		20			0.978757597683		0.974630892312
	20		30			0.99360541291		0.99584049261
	30		5			0.850375195331		0.780440208053
	30		10			0.91116554994		0.864104617442
	30		15			0.941101177408		0.901174769283
	30		20			0.975973632328		0.969110798392
	30		30			0.996748484908		0.999758981815

### SCIKIT LEARN

In [ ]:
# Import SciKitLearn
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GMM

# Regression models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso, MultiTaskLassoCV, LassoLarsCV, LassoLarsIC 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV as CV

# Other packages
import time
import numpy as np
import pandas as pd
import re
import scipy as sp
import matplotlib.pyplot as plt
from numpy import concatenate
from pandas import DataFrame, Series, read_csv, scatter_matrix

# Install packages
!pip install keras



In [ ]:
#  Mean Absolute Error Function

def get_mean_abs_error(y_true, y_pred, weights):
  return np.exp(np.mean(np.mean(np.abs(y_true - y_pred), axis=1)))

In [ ]:
user_session_with_window_bucketized_descr_df.show(5)

In [ ]:
userSession_df.show(5,False)

In [ ]:
int("user_000055".split("_")[1].lstrip("0"))

In [ ]:
def get_user_num(userid):
    return int(userid.split("_")[1].lstrip("0"))


In [ ]:
get_user_num_udf = udf(lambda userid: get_user_num(userid), IntegerType())

In [ ]:
userSession_df_ = userSession_df.withColumn('usernum', get_user_num_udf('userid'))

In [ ]:
userSession_df_.show(5)

In [ ]:
userSession_df.printSchema()

In [ ]:
country_ids = the_model_df_.select('country','country_index').distinct().toPandas().set_index('country').T.to_dict('list')

In [ ]:
userSession_df_.show(5)

In [ ]:
int(country_ids['Turkey'][0])

In [ ]:
def get_country_id(country):
    try:
        return int(country_ids[country][0])
    except:
        return 9999999

In [ ]:
get_country_id_udf = udf(lambda country: get_country_id(country), IntegerType())

In [ ]:
userSession_df_ = userSession_df_.withColumn('country_id', get_country_id_udf('country'))

In [ ]:
userSession_df_.show(5, False)

In [ ]:
#data = userSession_df_.select('usernum','session_id','day_of_week_index','time_of_day_index','session_length_1','session_length_2','session_length_3','session_length_4','session_length_5',"absence_time",'days_since_epoch','is_holiday','session_length').dropna().toPandas()
data = userSession_df_.select('usernum','session_id','day_of_week_index','time_of_day_index',"genre_10_freq","genre_9_freq","genre_8_freq","genre_7_freq","genre_6_freq","genre_5_freq","genre_4_freq","genre_3_freq","genre_2_freq","genre_1_freq","absence_time",'days_since_epoch','is_holiday','session_length').toPandas()

In [ ]:
data.head()

In [ ]:
#model_df = data[["userid","age","gender","country_index", "day_of_week", "time_of_day",'session_length_1','session_length_2','session_length_3','session_length_4','session_length_5','session_length',"is_holiday","session_length"]]
model_df = data

In [ ]:
model_df.head()

### TENSORFLOW

In [ ]:
from sklearn.utils import shuffle
from sklearn import preprocessing

In [ ]:
model_spark = spark.createDataFrame(model_df)
model_spark.show()

In [ ]:
type(model_spark)

In [ ]:
session_length_bucketizer = QuantileDiscretizer(numBuckets=num_buckets, inputCol="session_length", outputCol="decile", relativeError=0.01, handleInvalid="error")
bucketizer = session_length_bucketizer.fit(model_spark)
model_bucketized_df = bucketizer.setHandleInvalid("skip").transform(model_spark).drop('session_length').withColumnRenamed('decile','session_length')
model_bucketized_df.show()

In [ ]:
unscaled_model_df = model_bucketized_df.select('usernum','session_id','day_of_week_index','time_of_day_index',"genre_10_freq","genre_9_freq","genre_8_freq","genre_7_freq","genre_6_freq","genre_5_freq","genre_4_freq","genre_3_freq","genre_2_freq","genre_1_freq","absence_time",'days_since_epoch','is_holiday',"session_length").toPandas().dropna()
unscaled_model_df[:5]

In [ ]:
type(unscaled_model_df)

In [ ]:
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(unscaled_model_df)
model_df = pd.DataFrame(scaled_df, columns=['usernum','session_id','day_of_week_index','time_of_day_index',"genre_10_freq","genre_9_freq","genre_8_freq","genre_7_freq","genre_6_freq","genre_5_freq","genre_4_freq","genre_3_freq","genre_2_freq","genre_1_freq","absence_time",'days_since_epoch',"is_holiday","session_length"])

In [ ]:
model_df[:5]

In [ ]:
#train cut point ~ 65% of samples
train_size = int(0.65*model_df.shape[0])

#dev cut point ~20% of samples
dev_size   = int(0.2*model_df.shape[0])

#test cut point ~15% of samples
test_size  = int(0.15%model_df.shape[0])
test_cut   = train_size + dev_size

In [ ]:
input_       = shuffle(model_df.reset_index())
input_data   = np.array(input_.iloc[:,1:-1]).astype(np.float32)
input_labels = np.array(input_.iloc[:,-1]).astype(np.float32)

In [ ]:
input_data[:5]

In [ ]:
input_labels[:5]

In [ ]:
deciles


In [ ]:
input_data.shape

In [ ]:
num_examples = train_size

In [ ]:
train_X = input_data[:train_size]
train_X[:5]

In [ ]:
train_X.shape

In [ ]:
train_X.dtype

In [ ]:
train_Y = input_labels[:train_size]
train_Y.shape

In [ ]:
dev_X = input_data[train_size:test_cut]
dev_X[:5]

In [ ]:
dev_Y = input_labels[train_size:test_cut]
dev_Y[:5]

In [ ]:
n_samples = train_X.shape[0]
n_samples

In [ ]:
num_features = train_X.shape[1]
num_features

### Logistic Regression with TensorFlow

In [ ]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn


In [ ]:
learning_rate   = 0.01
training_epochs = 100
batch_size      = 1000
display_step    = 10

In [ ]:
# tf Graph Input
X = tf.placeholder(tf.float32, [batch_size, num_features])
Y = tf.placeholder(tf.float32, [batch_size,])

# Set model weights
W = tf.Variable(tf.zeros([num_features, batch_size]))
b = tf.Variable(tf.zeros([batch_size]))

In [ ]:
with tf.name_scope('prediction'):
    pred = tf.nn.softmax(tf.matmul(X, W) + b)
    tf.summary.histogram('predictions', pred)

with tf.name_scope('cost'):
    cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(pred), reduction_indices=1))
    tf.summary.histogram('cost', cost)
    
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

In [ ]:
def get_next_batch(i, b_size,data,labels):
    start = i*b_size
    end = start + b_size
    return data[start:end], labels[start:end]

In [ ]:
[ type(x) for x in get_next_batch(0,batch_size, train_X, train_Y)]

In [ ]:
n_samples

In [ ]:
dev_X.shape

In [ ]:
dev_Y.shape

In [ ]:
!pwd

In [ ]:
!tensorboard --logdir=/data_data/session_length/logs/

In [ ]:
#!mkdir ./logs

### LSTM Classifier: Train classifier to predict decile that a session will fall into

In [ ]:
# Start training
with tf.Session() as sess:
    sess.run(init)
    
    with tf.name_scope('train'):
        for epoch in range(training_epochs):
            avg_cost = 0
            total_batch = int(n_samples/batch_size)

            for i in range(total_batch):
                #print("ITERATION:\t{}\tBATCH_SIZE:\t{}".format(i,batch_size))
                batch_xs, batch_ys = get_next_batch(i,batch_size, train_X, train_Y)

                _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
                #print("cost:\t{}".format(c))
                avg_cost += c / total_batch

            if (epoch+1) % display_step == 0:
                print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

                tf.summary.scalar('average_cost', avg_cost)
            
        print("Optimization Finished!")
        
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print("Accuracy:\t{}".format(accuracy))
        tf.summary.scalar('accuracy', accuracy)
    
    devX = dev_X[:batch_size]
    devY = dev_Y[:batch_size]
    
    print("Accuracy:", accuracy.eval({X: devX, Y: devY}))
    
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('/data_data/session_length/logs/train', sess.graph)
    test_writer  = tf.summary.FileWriter('/data_data/session_length/logs/test')

In [ ]:
!whoami

### LSTM Classifier: Using Keras

In [ ]:
from keras import models
from keras import layers

In [ ]:
train_X[:5]

In [ ]:
train_Y[:5]

In [ ]:
len(train_Y)

In [ ]:
model = models.Sequential()
model.add(layers.Dense(100, activation='relu', input_shape=(len(train_Y,))))

### LSTM Classifier Part II

In [ ]:
# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 100
display_step = 200

# Network Parameters
num_input = 7 
timesteps = 28 
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [ ]:
hidden_layer_size = 100

num_classes       = 10

_seq_len          = n_samples

_labels           = deciles

time_steps        = 10

embed_dimension   = 50

In [ ]:
_inputs   = tf.placeholder(tf.int32,   shape=[batch_size, time_steps])
_labels   = tf.placeholder(tf.float32, shape=[batch_size, num_classes])
_size     = 10
_seqlens  = tf.placeholder(tf.int32, shape=[batch_size])

In [ ]:
with tf.name_scope('embeddings'):
    embeddings = tf.Variable( tf.random_uniform([_size, embed_dimension], -1.0, 1.0), name='embedding')
    embed      = tf.nn.embedding_lookup(embeddings, _inputs)

In [ ]:
embeddings.get_shape()

In [ ]:
embed.get_shape()

In [ ]:
tf.reset_default_graph()

with tf.variable_scope('lstm'):
    
    lstm_cell       = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size, forget_bias=1.0)
    
    print("LSTM output_size: \t{}\tinput_spec:\{}".format(lstm_cell.output_size,lstm_cell.input_spec))
    
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, embed, sequence_length = _seqlens, dtype=tf.float32)
    
    weights         = { 'linear_layer': tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], mean=0, stddev=0.01)) }
    
    biases          = { 'linear_layer': tf.Variable(tf.truncated_normal([num_classes], mean=0, stddev=0.01))}
    
    final_output    = tf.matmul(states[1], weights["linear_layer"]) + biases["linear_layer"]
    
    softmax         = tf.nn.softmax_cross_entropy_with_logits(logits = final_output, labels = _labels)
    
    cross_entropy   = tf.reduce_mean(softmax)
    
    

In [ ]:
train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

correction_prediction = tf.equal(tf.argmax(_labels, 1), tf.argmax(final_output, 1))

accuracy   = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    for step in range(1000):
        
        x_batch, y_batch = get_next_batch(step, batch_size, train_X, train_Y)
    
        if step % 100 == 0:

            acc = sess.run(accuracy,feed_dict={inputs:x_batch, _labels:y_batch})

            print("Accuracy at %d: %.5f" % (step, acc))
            
            
    for test_batch in range(5):
        
        x_tes, y_test = get_next_batch(test_batch, batch_size, train_X, train_Y)
        
        batch_pred, batch_acc = sess.run([tf.argmax(final_output,1), accuracy], feed_dict={_inputs:x_test, _labels:y_test})
        
        print("Test batch accuracy %d: %.5f" % (test_batch, batch_acc))

    ouput_example  = session.run([outputs], feed_dict={_inputs:x_test, _labels:y_test})
    
    states_example = sess.run([states[1]], feed_dict={_inputs:x_test, _labels:y_test})
        
    

### CNN with TensorFlow